# Determing Extent of Water Bodies<img align="right" src="../Supplementary_data/DE_Africa_Logo_Stacked_RGB_small.jpg">

* **Products used:** 
[ga_ls8c_wofs_2](https://explorer.digitalearth.africa/ga_ls8c_wofs_2), 
[ls8_usgs_sr_scene](https://explorer.digitalearth.africa/ls8_usgs_sr_scene)




## Background
The United Nations have prescribed 17 "Sustainable Development Goals" (SDGs). This notebook attempts to monitor SDG Indicator 6.6.1 - change in the extent of water-related ecosystems. Indicator 6.6.1 has 4 sub-indicators:
>    i. The spatial extent of water-related ecosystems <br>
>    ii. The quantity of water contained within these ecosystems <br>
>    iii. The quality of water within these ecosystems <br>
>    iv. The health or state of these ecosystems <br>

This notebook primarily focuses on the first sub-indicator - spatial extents.

The [Water Observations from Space (WOfS)](https://www.ga.gov.au/scientific-topics/community-safety/flood/wofs/about-wofs) product shows water observed for Landsat-8 image for selected areas of Africa and it will be used to determine the spatial extent of water.


## Description
The notebook demonstrates how to:

1. Load WOFL data for a given location and time period
2. Set up a Dask cluster
3. Generate a time series
4. Generate the Map for Two time period
5. Create animation for the Product
6. Compare Two Time Periods

***

## Getting started
To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

In [ ]:
%matplotlib inline

import datacube
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys, warnings
import xarray as xr
from IPython.display import Image
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch

sys.path.append('../Scripts')

from deafrica_datahandling import mostcommon_crs, load_ard, wofs_fuser
from deafrica_bandindices import calculate_indices
from deafrica_plotting import display_map, rgb, animated_timeseries, xr_animation
from deafrica_dask import create_local_dask_cluster



### Connect to the datacube

Activate the datacube database, which provides functionality for loading and displaying stored Earth observation data.

In [ ]:
dc = datacube.Datacube(app='water_extent')

## Set up a Dask cluster

Dask can be used to better manage memory use and conduct the analysis in parallel. 
For an introduction to using Dask with Digital Earth Africa, see the [Dask notebook](../Beginners_guide/06_Parallel_processing_with_dask.ipynb).

>**Note**: We recommend opening the Dask processing window to view the different computations that are being executed; to do this, see the *Dask dashboard in DE Africa* section of the [Dask notebook](../Beginners_guide/06_Parallel_processing_with_dask.ipynb).

To activate Dask, set up the local computing cluster using the cell below.

In [ ]:
create_local_dask_cluster()

### Analysis parameters

The following cell sets the parameters, which define the area of interest and the length of time to conduct the analysis over.

The parameters are:

* `lat`: The central latitude to analyse (e.g. 10.338).
* `lon` : The central longitude to analyse (e.g. -1.055).
* `lat_buffer` : The number of degrees to load around the central latitude.
* `lon_buffer` : The number of degrees to load around the central longitude.

* `Start_year and end_year`: The date range to analyse (e.g. `('2015', '2019')`).

**If running the notebook for the first time**, keep the default settings below.
This will demonstrate how the analysis works and provide meaningful results.
The example covers part of the Lake Sulunga. Tanzania. 


In [ ]:
# Define the area of interest
lat = -6.0873
lon = 35.1817

lat_buffer = 0.19
lon_buffer = 0.2

# Combine central lat,lon with buffer to get area of interest
lat_range = (lat - lat_buffer, lat + lat_buffer)
lon_range = (lon - lon_buffer, lon + lon_buffer)

# Define the start year and end year
start_year = 2013
end_year = 2019

## View the Area of Interest on an Interative map
The next cell will display the selected area on an interactive map.
The red border represents the area of interest of the study.
Zoom in and out to get a better understanding of the area of interest.
Clicking anywhere on the map will reveal the latitude and longitude coordinates of the clicked point.

In [ ]:
display_map(lon_range, lat_range)

## Load and view Landsat data

The code below will create a query dictionary for our region of interest, find the correct `crs` using the `mostcommon_crs function` for the area of interest, and then load the WOFs product with `dc.load` function.
For more infmation loading data, see the [Using load_notebook](../Beginners_guide/03_Loading_data.ipynb).

In [ ]:
#Create a query object
query = {
    'x': lon_range,
    'y': lat_range,
    'group_by': 'solar_day',
    'resolution': (-30, 30),
    'align': (15, 15),
    'time': (start_year, end_year)
}


# find the right crs for the location
crs = mostcommon_crs(dc=dc, product='ga_ls8c_wofs_2', query=query)

# Load available data from all three Landsat satellites
ds = dc.load(product=["ga_ls8c_wofs_2"],
             output_crs=crs,
             dask_chunks={'time': 1, 'x': 2000, 'y': 2000},
             fuse_func=wofs_fuser,
             **query
            )

ds_landsat = dc.load(product=['ls8_usgs_sr_scene'],
                     measurements=['red', 'green', 'blue'],
                     dask_chunks={'time': 1, 'x': 2000, 'y': 200},
                     like=ds)



## Calculate the area per pixel
The number of pixels can be used for the area of the waterbody if the pixel area is known. 
Run the following cell to generate the necessary constants for performing this conversion.

In [ ]:
pixel_length = query["resolution"][1]  # in metres
m_per_km = 1000  # conversion from metres to kilometres
area_per_pixel = pixel_length**2 / m_per_km**2

## Defining the threshold value
The next cell calcualates the threshold value to prevent partial image scene, missing data and also reduce the cloud cover.

In [ ]:
# More than 75% of pixels need to be classified as land or water
valid_threshold = (ds.x.size * ds.y.size) * 0.75 # Needs 75% to be water or land

## Calculating the extent of water
The following cell calculate the following 
1. Classify pixels as True if they are land or water
2. Count all pixels classified as land and water in each timestep
3. Apply the threshold value to remove unwanted scenes
4. Calculate area of water extent

In [ ]:
# Classify pixels as True if they are land or water; classifies as False otherwise
ds_land_water = ds.water.isin([0, 128])

# Count all pixels classified as land and water in each timestep
ds_land_water_count = ds_land_water.sum(dim=['x', 'y'])

# Only keep pixels if there are more land+water pixels than the valid threshold
# Apply this to the original dataset using .where to get WOfS pixel values
ds_valid = ds.where(ds_land_water_count > valid_threshold)

# remove any timesteps that did not meet the valid condition
ds_valid_dropna = ds_valid.dropna(dim='time', how='all')

# Only show water pixels
ds_valid_water = ds_valid_dropna.water == 128

# Calculate area of water pixels
ds_valid_water_area = ds_valid_water.sum(dim=['x', 'y']) * area_per_pixel

## Plot Monthly Time Series from the `Start year` to `End year`

In [ ]:
plt.figure(figsize=(18, 4))
ds_valid_water_area.plot(marker='o', color='#9467bd')
plt.title(f'Time Series Analysis of water observed area from {start_year} to {end_year}')
plt.xlabel('Dates')
plt.ylabel('Waterbody area (km$^2$)')
plt.tight_layout()

## Determine Minimum and Maximum Water Extent
The next cell extract the Minimum and Maximum extent of water from the dataset using the `min` and `max` functions

In [ ]:
min_water_area_date, max_water_area_date =  min(ds_valid_water_area), max(ds_valid_water_area)

A new dataArray is created to store the new date from the `min_water_area_date` and `max_water_area_date` from the above function

In [ ]:
time_xr = xr.DataArray([min_water_area_date.time.values, max_water_area_date.time.values], dims=["time"])

## Plotting
Plot water classified pixel for the two dates where we have the minimum and maximum surface water extent.

In [ ]:
ds_valid_water.sel(time=time_xr).plot.imshow("x","y",col="time", col_wrap=2, robust=True, figsize=(14, 6), cmap='jet_r')

Plot the Equivalent landsat  (water classified pixel) for the two dates where we have the minimum and maximum surface water extent.

In [ ]:
ds_landsat_valid = ds_landsat.sel(time=time_xr)
rgb(ds_landsat_valid, col='time') 

## Animating time series
In the next cell, we plot the dataset we loaded above as an animation GIF, using the [`xr_animation`](../Frequently_used_code/Animated_timeseries.ipynb) function.
The `output_path` will be saved in the directory where the script is found and you can change the names to prevent files overwrite.
A legendbar is added below the output image , where `0 = land and 128 = water, the middle color is the clouds cover`.

In [ ]:
xr_animation(ds=ds_valid_dropna, 
             output_path='water_extent.gif',
             bands ='water',
             interval=1000, 
             width_pixels=300,
             show_colorbar=True,
             imshow_kwargs={'cmap': 'RdYlBu'},
            )

# Plot animated gif
plt.close()
Image(filename='water_extent.gif')

## Compare Two Time Periods - a Baseline and an Analysis
The following cells determine the maximum extent of water for two different years. 
* `baseline_year` : The baseline year for the analysis
* `analysis_year` : The year to compare to the baseline year

* The maximum water extent for the two years are determine from the datatset `ds_valid_water_area`

In [ ]:
baseline_year = 2016
analysis_year = 2019

baseline_ds, analysis_ds = max(ds_valid_water_area[ds_valid_water_area['time.year'] == baseline_year]) , max(ds_valid_water_area[ds_valid_water_area['time.year'] == analysis_year])

A new dataArray is created to store the new date from the `maximum water extent` for the two years

In [ ]:
time_xr = xr.DataArray([baseline_ds.time.values, analysis_ds.time.values], dims=["time"])

## Plotting
Plot water extent of the WOFS product for the two years.

In [ ]:
ds_valid_water.sel(time=time_xr).plot.imshow("x","y",col="time", col_wrap=2, robust=True, figsize=(14, 6), cmap='jet_r')

Plot the Equivalent landsat for the two dates

In [ ]:
rgb(ds_landsat.sel(time=time_xr), col='time') 

## Calculating the change for the two periods
The cells below calculate the amount of water gain, loss and stable for the two periods

In [ ]:
# The two period Extract the two periods(Baseline and analysis) dataset from
ds_selected = ds_valid_water.sel(time=time_xr)

#The dataset array is transform to 1 and 0 using the `astype(int)` function.
analyse_total_value = ds_selected[1].astype(int)
change = analyse_total_value - ds_selected[0].astype(int)

#The various scenarios are extracted from the `change` variable for the tw years
#Water gain  is 1
water_appeared = change.where(change == 1)

#Permanent water = 0
permanent_water = change.where((change == 0) & (analyse_total_value == 1))

#Permanent Land = 0
permanent_land = change.where((change == 0) & (analyse_total_value == 0))

#Water loss = -1
water_disappeared = change.where(change == -1)

The cell below calculate the area of water extent for water_loss, water_gain, permanent water and land 

In [ ]:
total_area = analyse_total_value.count().values * area_per_pixel
water_apperaed_area = water_appeared.count().values * area_per_pixel
permanent_water_area = permanent_water.count().values * area_per_pixel
permanent_land_area = permanent_land.count().values * area_per_pixel
water_disappeared_area = water_disappeared.count().values * area_per_pixel

The cell below calculate the percentage of area of water extent for water_loss, water_gain, permanent water and land 

In [ ]:
# Percentage Change in water extent
percentage_water_gain = round((water_apperaed_area /  total_area) * 100 , 2)
percentage_water_loss = round(water_disappeared_area /  total_area * 100 , 2)
percentage_permanent_water = round(permanent_water_area /  total_area * 100 , 2)
percentage_permanent_land = round(permanent_land_area /  total_area * 100, 2)

## Plotting
The water variables are plotted to visualised the result

In [ ]:
water_appeared_color = 'Green'
water_disappeared_color = 'Yellow'
stable_color = 'Blue'
land_color = 'Brown'

ds_valid_water.sel(time=time_xr[1]).plot.imshow(cmap='jet_r', add_colorbar=False, add_labels=False, size=6)
water_appeared.plot.imshow(cmap=ListedColormap([water_appeared_color]), add_colorbar=False, add_labels=False)
water_disappeared.plot.imshow(cmap=ListedColormap([water_disappeared_color]), add_colorbar=False, add_labels=False)

plt.legend(
        [Patch(facecolor=stable_color), Patch(facecolor=water_disappeared_color), Patch(facecolor=water_appeared_color),   Patch(facecolor=land_color)], 
        [f'Water to Water {percentage_permanent_water}%', f'Water to No Water {percentage_water_loss}%' , f'No Water to Water: {percentage_water_gain}%',  
         f'No Water to No water {percentage_permanent_land}%'],
        loc = 'lower left'
    )

## Next steps


Return to the "Analysis parameters" section, modify some values (e.g. `latitude`, `longitude`, `start_year`, `end_year`) and re-run the analysis.
You can use the interactive map in the "View the selected location" section to find new central latitude and longitude values by panning and zooming, and then clicking on the area you wish to extract location values for.
You can also use Google maps to search for a location you know, then return the latitude and longitude values by clicking the map.

Change the year aslo in "Compare Two Time Periods - a Baseline and an Analysis" section, (e.g. `base_year`, `analyse_year`) and re-run the analysis.

***

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Last modified:** July 2020

**Compatible datacube version:** 

In [ ]:
print(datacube.__version__)

## Tags
Browse all available tags on the DE Africa User Guide's [Tags Index](https://) (placeholder as this does not exist yet)